This demo is from demo 1 in [1].

In [1]:
from SOSPy import *
from sympy import symbols
import numpy as np

## Sum of Squares Test

Testing if a polynomial $p(x)$ is nonnegative for all $x \in \mathbb{R}^n$ can be relaxed to the problem of checking if $p(x)$ is an SOS.

$p(x)$ is an SOS indicating $p(x) = Z^T(x)QZ(x)$. In this problem, $p(x) = 2x_1^4 + 2x_1^3x_2-x_1^2x_2^2+5x_2^4$, the highest degree of monomials in $p(x)$ is 4 and the lowest degree of monomials is also 4, so it is reasonable to choose the highest and lowest degree in $Z(x)$ to be both 2. 

Therefore, $p(x) = \begin{bmatrix} x_1^2 & x_1x_2 & x_2^2 \end{bmatrix} Q \begin{bmatrix} x_1^2 \\ x_1x_2 \\ x_2^2 \end{bmatrix}$. 

Let $Q = \begin{bmatrix} q_1 & q_2 & q_3 \\ q_4 & q_5 & q_6 \\ q_7 & q_8 & q_9 \end{bmatrix}$. Since $Q$ is symmetric, $q_2 = q_4, q_3 = q_7, q_6 = q_8$, but we don't substitute them yet. (In code, this step is automatically done in MOSEK solver model.)

Since $\begin{bmatrix} x_1^2 & x_1x_2 & x_2^2 \end{bmatrix} Q \begin{bmatrix} x_1^2 \\ x_1x_2 \\ x_2^2 \end{bmatrix} = 2x_1^4 + 2x_1^3x_2-x_1^2x_2^2+5x_2^4$, we can easily get an equation: 
$$ \begin{bmatrix} 0&0&0&0&0&0&0&0&1\\0&0&0&0&0&1&0&1&0\\0&0&1&0&1&0&1&0&0\\0&1&0&1&0&0&0&0&0\\1&0&0&0&0&0&0&0&0 \end{bmatrix} \begin{bmatrix} q_1\\q_2\\q_3\\q_4\\q_5\\q_6\\q_7\\q_8\\q_9 \end{bmatrix} = \begin{bmatrix} 5\\0\\-1\\2\\2 \end{bmatrix}$$. 

We can write this in the **SDP primal form** that MOSEK can solve:

\begin{align*}
    \text{No Objective Function}\quad &\\
    \text{subject to}\quad & \left\langle\begin{bmatrix} 0&0&0\\0&0&0\\0&0&1 \end{bmatrix}, \bar{\mathbf{X}} \right\rangle = 5\\
    & \left\langle\begin{bmatrix} 0&0&0\\0&0&1\\0&1&0 \end{bmatrix}, \bar{\mathbf{X}} \right\rangle = 0\\
    & \left\langle\begin{bmatrix} 0&0&1\\0&1&0\\1&0&0 \end{bmatrix}, \bar{\mathbf{X}} \right\rangle = -1\\
    & \left\langle\begin{bmatrix} 0&1&0\\1&0&0\\0&0&0 \end{bmatrix}, \bar{\mathbf{X}} \right\rangle = 2\\
    & \left\langle\begin{bmatrix} 1&0&0\\0&0&0\\0&0&0 \end{bmatrix}, \bar{\mathbf{X}} \right\rangle = 2\\
    & \quad \bar{\mathbf{X}} \succeq 0
\end{align*}

where $\bar{\mathbf{X}} = Q = \begin{bmatrix} q_1 & q_2 & q_3 \\ q_4 & q_5 & q_6 \\ q_7 & q_8 & q_9 \end{bmatrix} \in \mathbf{S}^3_+$.
Since $\bar{\mathbf{X}}$ is symmetric, there are 6 variables. 

If there is a solution, then this problem is feasible, which indicates $p(x) \geq 0$. The following are about code.

First, we declare the symbols $x_1$ and $x_2$, and symbolic expression $p(x)$.

In [2]:
[x1,x2] = symbols("x1,x2")
vartable = [x1,x2]
p = 2*x1**4 + 2*x1**3*x2 - x1**2*x2**2 + 5*x2**4

Then, we can initialize the sum of squares program. 

In [3]:
prog = sosprogram(vartable)

Installed SDP solvers:  ['MOSEK', 'CVXOPT', 'SCS', 'SDPA']


Now, **prog** contains the information of scalar variables $x_1$ and $x_2$.

In [4]:
print('Scalar Variables:', prog.symvartable)

Scalar Variables: [x1, x2]


We can now add Sum of Square inequality to the program.

In [5]:
prog = sosineq(prog,p)

Information about this inequality is stored in **prog.expr**

In [6]:
prog.expr

{'num': 1,
 'type': {0: 'ineq'},
 'At': {0: <0x4 sparse matrix of type '<class 'numpy.float64'>'
  	with 0 stored elements in Compressed Sparse Row format>},
 'b': {0: <4x1 sparse matrix of type '<class 'numpy.float64'>'
  	with 4 stored elements in Compressed Sparse Row format>},
 'Z': {0: <4x2 sparse matrix of type '<class 'numpy.intc'>'
  	with 6 stored elements in Compressed Sparse Row format>},
 'multipart': {}}

We can try to solve the program as a feasibility problem. If the result is feasible, then $p(x)$ is an SOS.

The default solver is **cvxopt**, you can use other solvers by specifying **option['solver'] = 'mosek'**, or **option['solver'] = 'scs'**.

In [7]:
options = {}
#options['solver'] = 'mosek'
#options['solver'] = 'scs'
prog = sossolve(prog,options,verbose=0)  # Set verbose=1 if you want to view the detailed process, rerun the whole demo


 Residual norm 1.0877919644084146e-15
cpusec: 0.004
iter: 5
status: optimal
pinf: 0.0
dinf: 0.0


The feasible ratio is 1 and status is "optimal", so $p(x)$ is an SOS, which indicates $p(x) \geq 0$ for all $x \in \mathbb{R}^2$

### Citation:

[1]: A. Papachristodoulou, J. Anderson, G. Valmorbida, S. Prajna, P. Seiler, P. A. Parrilo, M. M. Peet, and D. Jagt, "4.1 Sum of Squares Test," in _Sum of Squares Optimization Toolbox for MATLAB, User’s guide_, Version 4.00, 2021, pp. 33-35.